In [3]:
import os
import sys
import torch
import torch.nn
import argparse
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from PIL import Image
from networks.resnet import resnet50

# file_dir = '/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/datasets/DiffusionForensics/dalle2/celebahq/1_fake/100.png'
weights_path = '/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/CNNDetection/checkpoints/test-4class-resnet-car-cat-chair-horse2024_08_09_22_33_48/model_epoch_last.pth'
# weights_path = "/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/deepfake_detection_models/NPR-DeepfakeDetection/NPR.pth"
file_dir = '/home/jovyan/shares/SR006.nfs2/almas_deepfake_detection/datasets/DiffusionForensics/dalle2/celebahq/0_real/000152.jpg'

parser = argparse.ArgumentParser(formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('-f','--file', default=file_dir)
parser.add_argument('-m','--model_path', type=str, default=weights_path)
parser.add_argument('-c','--crop', type=int, default=None, help='by default, do not crop. specify crop size')
parser.add_argument('--use_cpu', action='store_true', help='uses gpu by default, turn on to use cpu')

opt = parser.parse_args()

model = resnet50(num_classes=1)
state_dict = torch.load(opt.model_path, map_location='cpu')
model.load_state_dict(state_dict['model'])
if(not opt.use_cpu):
  model.cuda()
model.eval()

# Transform
trans_init = []
if(opt.crop is not None):
  trans_init = [transforms.CenterCrop(opt.crop),]
  print('Cropping to [%i]'%opt.crop)
else:
  print('Not cropping')
trans = transforms.Compose(trans_init + [
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

img = trans(Image.open(file_dir).convert('RGB'))

with torch.no_grad():
  in_tens = img.unsqueeze(0)
  if not opt.use_cpu:
    in_tens = in_tens.cuda()
  prob = model(in_tens).sigmoid().item()

print('probability of being synthetic: {:.2f}%'.format(prob * 100))


/tmp/ipykernel_1605655/1128297007.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(opt.model_path, map_location='cpu')


Not cropping
probability of being synthetic: 0.00%
